# Experiments

Here, we compare standard baselines to imitation learning and RL-based methods. Please read the readme first.

In [ ]:
#First, we import our libraries and define the neural net model used for DQN. 
#We specify store hyperparameters then build the training function for our agent

import torch.nn as nn
import torch
from retail import retail
import torch.distributions as d
import torch.nn.functional as F
import numpy as np
bucket_customers = torch.tensor([800.,400.,500.,900.])

from rlpyt.algos.dqn.dqn import DQN
from rlpyt.agents.pg.categorical import CategoricalPgAgent
from rlpyt.agents.dqn.dqn_agent import DqnAgent
from rlpyt.samplers.serial.sampler import SerialSampler
from rlpyt.agents.dqn.dqn_agent import DqnAgent
from rlpyt.runners.minibatch_rl import MinibatchRlEval
from rlpyt.utils.logging.context import logger_context
from rlpyt.samplers.parallel.cpu.sampler import CpuSampler
from rlpyt.algos.pg.ppo import PPO
from rlpyt.agents.pg.gaussian import GaussianPgAgent
torch.set_default_tensor_type(torch.FloatTensor)

class DQNModel(torch.nn.Module):
    def __init__(
            self,
            input_size,
            conv_size,
            hidden_sizes,
            n_kernels,
            output_size=None,  
            nonlinearity=torch.nn.SELU,  
            ):
        super().__init__()
        self._conv_size = conv_size
        self._char_size = input_size-conv_size
        self.normL = torch.nn.LayerNorm(input_size)
        self._transf_in_size = input_size-conv_size+n_kernels
        self.activ = nn.SELU()
        if isinstance(hidden_sizes, int):
            hidden_sizes = [hidden_sizes]
        hidden_layers = [torch.nn.Linear(n_in, n_out) for n_in, n_out in
            zip([self._transf_in_size] + hidden_sizes[:-1], hidden_sizes)]
        sequence = list()
        for layer in hidden_layers:
            sequence.extend([layer, nonlinearity(), torch.nn.LayerNorm(layer.out_features)])
        if output_size is not None:
            last_size = hidden_sizes[-1] if hidden_sizes else input_size
            sequence.append(torch.nn.Linear(last_size, output_size))
        self.convs = torch.nn.Conv1d(1, out_channels = n_kernels, kernel_size = 1)
        self.model = torch.nn.Sequential(*sequence)
        self._output_size = (hidden_sizes[-1] if output_size is None
            else output_size)

    def forward(self, o, action, reward):
        oprime = self.normL(o)
        stock, characteristics = o.split([self._conv_size,self._char_size],-1)
        stock_summary = self.convs(stock.view(-1,1,self._conv_size)).mean(2)
        if len(characteristics.shape)>2:
            stock_summary.unsqueeze_(1)
        summarized_input = torch.cat((self.activ(stock_summary), characteristics),-1)
        out = self.model(summarized_input)
        return out
    @property
    def output_size(self):
        return self._output_size

kwargsStore= {'assortment_size': 1, 'utility_weights' :{'alpha': 1., 'beta': 1., 'gamma': 1.},
             'max_stock': 1000, 'forecastVariance' :0.000, 'horizon': 730, 'lead_time': 1}
kwargsModel = {'input_size': 1006, 'hidden_sizes': [100,200, 500, 1000],  'conv_size': 1000, 'n_kernels': 50}

def build_and_train(run_ID=1, cuda_idx=None, env_kwargs = None, mid_batch_reset = False, 
                    n_parallel = 1, model_kwargs=None, initial_model_state_dict = None):
    affinity = dict(workers_cpus=list(range(n_parallel)))
    sampler = CpuSampler(
        EnvCls= retail.StoreEnv,
        env_kwargs = env_kwargs,
        eval_env_kwargs = env_kwargs,
        batch_T=20,
        batch_B=2,
        max_decorrelation_steps=10,
        eval_n_envs=1,
        eval_max_steps=int(20e4),
        eval_max_trajectories=24
    )
    algo = DQN(learning_rate=5e-5,discount=.99, replay_ratio=8, batch_size = 16) 
    agent = DqnAgent(ModelCls=DQNModel, model_kwargs=model_kwargs)
    runner = MinibatchRlEval(
        algo=algo,
        agent=agent,
        sampler=sampler,
        n_steps=10e4,
        log_interval_steps=1e3,
        affinity=affinity )
    
    name = "dqn"
    log_dir = "experiments"
    config = None
    runner.train()
    dic = agent.state_dict()
    torch.save(dic,'dqn-lt1.pt')

In [ ]:
#Next, we simply train our DQN agent.

build_and_train(
        run_ID=10,
        env_kwargs = kwargsStore,
        model_kwargs = kwargsModel)

In [ ]:
# We define the network for PPO, then its parameter and training function
class NewPPO(torch.nn.Module):
    def __init__(
            self,
            input_size,
            conv_size,
            hidden_sizes,
            n_kernels,
            output_size=None,  
            nonlinearity=torch.nn.SELU,  
            ):
        super().__init__()
        self._conv_size = conv_size
        self._char_size = input_size-conv_size
        self.normL = torch.nn.LayerNorm(input_size)
        self._transf_in_size = input_size-conv_size+n_kernels
        self.activ = nn.SELU()
        if isinstance(hidden_sizes, int):
            hidden_sizes = [hidden_sizes]
        hidden_layers = [torch.nn.Linear(n_in, n_out) for n_in, n_out in
            zip([self._transf_in_size] + hidden_sizes[:-1], hidden_sizes)]
        sequence = list()
        for layer in hidden_layers:
            sequence.extend([layer, nonlinearity(), torch.nn.LayerNorm(layer.out_features)])
        if output_size is not None:
            last_size = hidden_sizes[-1] if hidden_sizes else input_size
            sequence.append(torch.nn.Linear(last_size, output_size))
        self.convs = torch.nn.Conv1d(1, out_channels = n_kernels, kernel_size = 1)
        self.model = torch.nn.Sequential(*sequence)
        self._output_size = (hidden_sizes[-1] if output_size is None
            else output_size)

    def forward(self, o, action, reward):
        oprime = self.normL(o)
        stock, characteristics = o.split([self._conv_size,self._char_size],-1)
        stock_summary = self.convs(stock.view(-1,1,self._conv_size)).mean(2)
        if len(characteristics.shape)>2:
            stock_summary.unsqueeze_(1)
        summarized_input = torch.cat((self.activ(stock_summary), characteristics),-1)
        out = self.model(summarized_input).squeeze()
        return out.split(1, dim = -1)
    @property
    def output_size(self):
        return self._output_size

    
kwargsStore= {'assortment_size': 1, 'symmetric_action_space': True,
             'max_stock': 1000, 'forecastVariance' :0.000, 'horizon': 730, 'lead_time': 1}
kwargsModel = {'input_size': 1006, 'hidden_sizes': [100,200,100], 
               'output_size': 3, 'conv_size': 1000, 'n_kernels': 50}

def build_and_train(run_ID=1, cuda_idx=None, env_kwargs = None, 
                    mid_batch_reset = False, n_parallel = 1,model_kwargs=None):
    affinity = dict(workers_cpus=list(range(n_parallel)))
    sampler = CpuSampler(
        EnvCls= retail.StoreEnv,
        env_kwargs = env_kwargs,
        eval_env_kwargs = env_kwargs,
        batch_T=20,
        batch_B=1,
        max_decorrelation_steps=20,
        eval_n_envs=100,
        eval_max_steps=int(10e4),
        eval_max_trajectories=20)
    algo = PPO(learning_rate = 1e-5, discount=.99, value_loss_coeff=.2, 
               normalize_advantage = True, ratio_clip = .5)
    agent = GaussianPgAgent(ModelCls=NewPPO, model_kwargs=kwargsModel)
    runner = MinibatchRlEval(
        algo=algo,
        agent=agent,
        sampler=sampler,
        n_steps=1e5,
        log_interval_steps=1e3,
        affinity=affinity
    )
    name = "pg"
    log_dir = "experiments"
    config = None
    runner.train()
    dic = agent.state_dict()
    torch.save(dic,'ppo-lt1.pt')-ppo-lt1.pt')

In [ ]:
#We then train our PPO based agent
build_and_train(
        run_ID=11,
        env_kwargs = kwargsStore,
        model_kwargs = kwargsModel)

In [ ]:
#We collect results for the forecast based policy
rewards_forecast_order = []
for i in range(100):
    ra = []
    done = False
    kwargsStore= {'assortment_size': 250, 'lead_time': 1,
             'max_stock': 1000, 'seed': i, 'forecastVariance' :0.000, 'horizon': 500}

    store = retail.StoreEnv(**kwargsStore)
    while not (done):
        customers = bucket_customers.sum()
        p = store.forecast.squeeze()
        std = torch.sqrt(customers*p+(1-p))
        order = F.relu(3*std+store.forecast.squeeze()*customers-store.get_full_inventory_position()).round()
        obs = store.step(order.numpy())
        ra.append(obs[1])
        done = obs[2]
    rewards_forecast_order.append(torch.stack(ra).mean())

In [ ]:
#Same for the fixed order rate

rewards_sq_policy = []
for i in range(100):
    ra = []
    done = False
    kwargsStore= {'assortment_size': 250, 'lead_time': 1,
             'max_stock': 1000, 'seed': i, 'forecastVariance' :0.000, 'horizon': 500}
    store = retail.StoreEnv(**kwargsStore)
    while not (done):
        order = F.relu(2*store.assortment.base_demand*bucket_customers.max()-store.get_full_inventory_position()).round()
        obs = store.step(order.numpy())
        ra.append(obs[1])
        done = obs[2]
    rewards_sq_policy.append(torch.stack(ra).mean())

In [ ]:
#Now, we recover our DQN network and collect results

kwargsModel = {'input_size': 1006, 'hidden_sizes': [100,200, 500, 1000],  'conv_size': 1000, 'n_kernels': 50}
newDic= torch.load('dqn-lt1.pt',map_location=torch.device('cpu'))
model = DQNModel(**kwargsModel)
model.load_state_dict(newDic['model'])


rewards_dqn = []

for i in range(100):
    print(i)
    ra = []
    done = False
    kwargsStore= {'assortment_size': 250, 'lead_time': 1,
             'max_stock': 1000, 'seed': i, 'forecastVariance' :0.000, 'horizon': 500}

    store = retail.StoreEnv(**kwargsStore)
    while not (done):
        mu = model(store.get_obs(),5, 10).max(1)[1]
        obs = store.step(mu.detach().numpy())
        ra.append(obs[1])
        done = obs[2]

    rewards_dqn.append(torch.stack(ra).mean())

In [ ]:
#Same for PPO

kwargsModel = {'input_size': 1006, 'hidden_sizes': [100,200,100], 
               'output_size': 3, 'conv_size': 1000, 'n_kernels': 50}
newDic= torch.load('ijcai_genplan-ppo-lt1.pt',map_location=torch.device('cpu'))
model = NewPPO(**kwargsModel)
model.load_state_dict(newDic)
rewards_ppo = []
for i in range(100):
    ra = []
    done = False
    kwargsStore= {'assortment_size': 250, 'lead_time': 1,'symmetric_action_space': True,
             'max_stock': 1000, 'seed': i, 'forecastVariance' :0.000, 'horizon': 500}

    store = retail.StoreEnv(**kwargsStore)
    while not (done):
        mu, sigma, value = model(store.get_obs(),5, 10)
        obs = store.step(mu.detach().numpy())
        ra.append(obs[1])
        done = obs[2]
    rewards_ppo.append(torch.stack(ra).mean())
    print(torch.stack(ra).mean())

In [ ]:
#We now move to imitation learning. We collect samples from a policy


actionList = []
observations = []
rewards = []
#Here, the seed ensure that we do not mix train and test
for i in range(101,1101):
    ra = []
    done = False
    kwargsStore= {'assortment_size': 1, 'lead_time': 1,
             'max_stock': 1000, 'seed': i, 'forecastVariance' :0.000, 'horizon': 500}
    store = retail.StoreEnv(**kwargsStore)
    while not (done):
        observations.append(store.get_obs())
        customers = bucket_customers.sum()
        p = store.forecast.squeeze()
        std = torch.sqrt(customers*p+(1-p))
        order = F.relu(3*std+store.forecast.squeeze()*customers-store.get_full_inventory_position()).round()
        obs = store.step(order.numpy())
        ra.append(obs[1])
        actionList.append(order)
        done = obs[2]
    rewards += ra

In [ ]:
from torch.utils.data import Dataset, DataLoader
class ClassicDataSet(Dataset):
    def __init__(self, x, y, r):
        self._x = x
        self._y = y
        self._r = r
        self._len = x.shape[0]
        
    def __len__(self):
        return(self._len)
    
    def __getitem__(self, index):
        return(self._x[index], self._y[index], self._r[index])


In [ ]:
y = torch.stack(actionList)
x = torch.stack(observations)
r = torch.stack(rewards)    
action_reaction = ClassicDataSet(x, y, r)

data_loader = DataLoader(action_reaction, batch_size=128,
                            shuffle=True, num_workers=2)


In [ ]:
#Now, we simply need to replicate this policy. The neural network used is the same as PPO
kwargsPPO = {'input_size': 1006, 'hidden_sizes': [100,200,100], 
               'output_size': 3, 'conv_size': 1000, 'n_kernels': 50}
newNet = NewPPO(**kwargsPPO)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(newNet.parameters(), lr=1e-3)
for epoch in range(20):
    # Training
    s = 0
    s2 = 0
    for data, target, reward in (data_loader):
        net_out = newNet(data,0,0)
        loss = criterion(net_out[0], target)
        s += loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(s)
name = 'imitationLearning.pt'
torch.save(newNet.state_dict(),name)

In [ ]:
#We now collect rewards from the imitating neural network

rewards_imitation = []
for i in range(100):
    print(i)
    ra = []
    done = False
    kwargsStore= {'assortment_size': 250, 'lead_time': 1,
             'max_stock': 1000, 'seed': i, 'forecastVariance' :0.000, 'horizon': 500}

    store = env_rlpyt.StoreEnv(**kwargsStore)
    while not (done):
        mu, sigma, value = newNet(store.get_obs(),5, 10)
        obs = store.step(mu.detach().numpy())
        ra.append(obs[1])
        done = obs[2]
    rewards_imitation.append(torch.stack(ra).mean())


In [ ]:
We can plot a histogram
import matplotlib.pyplot as plt
import seaborn as sns
sns.kdeplot(torch.stack(rewards_dqn).numpy())

We can now save the results. It's easy to compute the mean or the std over the obtained sample.

In [ ]:
baseline_csv = torch.stack((torch.stack(rewards_sq_policy), torch.stack(rewards_forecast_order))).numpy()
rl_csv = torch.stack((torch.stack(rewards_dqn), torch.stack(rewards_ppo), torch.stack(rewards_imitation))).numpy()
np.savetxt("baseline_results.csv", baseline_csv, delimiter=",")
np.savetxt("rl_results.csv", rl_csv, delimiter=",")

